### 1.Build the docker image required for training.

We will be using [Neuronx-nemo-megatron](https://github.com/aws-neuron/neuronx-nemo-megatron) to pretrain/fine tune llama 7b model using AWS Trn1 (trn1.32xlarge) instance. Neuronx-nemo-megatron is a custom open source library that includes modfied version of the open-source packages Nemo and Apex that have been adapted for use with AWS Neuron and AWS Trn1 instances. 

We will start with building an custom docker image extending from [AWS Neuron Deep Learning container](https://github.com/aws/deep-learning-containers/blob/master/available_images.md#neuron-containers) for SageMaker and installing the Neuronx-nemo-megatron library. Please make sure your host instance has docker setup and have necessary permissions to create and push images to Amazon ECR.

Note: You can use SageMaker Notebook instances to build the image as a quick mechanism as it comes with Docker preinstalled. 

Run the below cells to trigger the docker image build process. 

We will start by installing/importing necessary packages. 

In [ ]:
%pip install --upgrade sagemaker

In [ ]:
%%time
import os

import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.huggingface import HuggingFace

role = (
    get_execution_role()
)  # provide a pre-existing role ARN as an alternative to creating a new role
print(f"SageMaker Execution Role: {role}")

client = boto3.client("sts")
account = client.get_caller_identity()["Account"]
print(f"AWS account: {account}")

session = boto3.session.Session()
region = session.region_name
print(f"AWS region: {region}")


In [ ]:
dlc_account_id = 763104351884  # account where AWS Deep learning containers are saved.

In [ ]:
image = (
    "nemo-megatron-neuron-sm-dlc" 
)
tag = "latest"

### Trigger the docker build

The below code cells will trigger the docker_build.sh script which builds the docker images and upload to ECR 

In [ ]:
!pygmentize build/docker_build.sh

In [ ]:
%%time

! sudo bash build/docker_build.sh $dlc_account_id $region $image $tag

In [ ]:
docker_image = f"{account}.dkr.ecr.{region}.amazonaws.com/{image}:{tag}"

In [ ]:
print(f"Training image URL {docker_image}")

In [ ]:
%store docker_image # Use store magic to save the docker image URL to use in subsequent notebooks.